<a href="https://colab.research.google.com/github/alex8937/introtodeeplearning_labs/blob/master/lab1%5Cmain_100_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/alex8937/introtodeeplearning_labs.git
% cd introtodeeplearning_labs
! git pull
% cd ..

fatal: destination path 'introtodeeplearning_labs' already exists and is not an empty directory.
/content/introtodeeplearning_labs
Already up to date.
/content


In [2]:
import tensorflow as tf
import numpy as np
import os
tf.enable_eager_execution()
text = ''
for file in os.listdir('introtodeeplearning_labs/lab1/data/'):
  if file.endswith('.txt'):
    text += open(os.path.join('introtodeeplearning_labs/lab1/data', file), encoding="utf8").read()
print('Entire corpus length:', len(text))

Entire corpus length: 6584432


In [3]:

vocab = sorted(set(text))
vocab_size = len(vocab)
print ('{} unique characters'.format(vocab_size))

char2idx = {c : idx for idx, c in enumerate(vocab)}
idx2char = np.array(vocab)

text_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_int)

seq_length = 100
n_samples = len(text) // seq_length
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)
dataset = sequences.map(lambda seq : (seq[:-1], seq[1:]))

batch_size = 64
n_mini_batches = n_samples // batch_size

#what if I don't do shuffle
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

rnn = tf.keras.layers.CuDNNLSTM


5169 unique characters


In [4]:

def build_model(vocab_size, embedding_size, rnn_units, batch_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_size,
                                        batch_input_shape = [batch_size, None]))
    model.add(rnn(rnn_units, 
                  return_sequences=True, 
                  recurrent_initializer='glorot_uniform',
                  stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

embedding_size = 256
rnn_units = 1024

model = build_model(vocab_size = vocab_size,
                    embedding_size=embedding_size,
                    rnn_units=rnn_units,
                    batch_size=batch_size)

#what if using softmax
def loss_fun(labels, logits):
    return tf.keras.backend.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = loss_fun)


checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                       save_weights_only=True)



Instructions for updating:
Colocations handled automatically by placer.


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
drive_weight_path = "/content/gdrive/My Drive/Colab Notebooks/model_weights_100.h5"
if os.path.exists(drive_weight_path):
  model.load_weights(drive_weight_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
history = model.fit(dataset.repeat(), 
                    epochs=1, 
                    steps_per_epoch=n_mini_batches, 
                    callbacks=[checkpoint_callback])
model.save_weights(drive_weight_path)

1027/1028 [============================>.] - ETA: 0s - loss: 2.3654WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1028/1028 [==============================] - 278s 270ms/step - loss: 2.3654


In [8]:
gen_model = build_model(vocab_size, embedding_size, rnn_units, batch_size=1)

gen_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

gen_model.build(tf.TensorShape([1, None]))



def generate_text(model, start_string, temperature = 1.0):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


print(generate_text(gen_model, start_string='小姐道：“我自幼清贫，山居简朴，这些珍物要来何用？可是岳父远道厚赐，若是不受，未免不恭。”'), 1)

小姐道：“我自幼清贫，山居简朴，这些珍物要来何用？可是岳父远道厚赐，若是不受，未免不恭。”
张无忌又道：“先前我二人走遍天下，原来你在这儿遇上周芷若的父亲杨康，黑暗中只有等人出去跟她成婚。那些乡下人马辛甚么啊啊？怎地跟我衣无生命？他所遇的矮江缥缈，玄净是半个病况而成名事。他压低了王氏贤侄的女儿，曾使吴三桂部下尽集于斯，保佑他为你报仇，却又何苦？”
张无忌红着脸道：“你瞧得明天明天，怎么事情好意？”将酒果向张淡月照着一行和尚冲进房来，心中微带着三分忌惮，何敢当即明白如此，二人终究不愿与她同流合调之人。
陆无双笑道：“你表妹信得过我么？”段誉道：“刚才阿朱必是阿朱这般的眼肿。”张无忌道：“她一切下令，事，到我家中便是。”又道：“大哥有先生的罪孽，我们只是尽力而为，引得菩萨保佑，这一阳指神功是全真派数百年来的上乘内功，武功不高，可是有模糊刺得井，再也顾念不语，伸出右手摇晃，蓦地里只在《笑傲江湖底皮紫凤冠英见到客人脸有忧色，显是亲热无比，心中恼怒，怒喝：“好，我便进来！”
慕容复喝道：“柳叶台，古人以胸面为我所刺，有甚么紧？你拔刀削去了一根手指，叫，何不便拔剑出去？”张无忌无法克制意根禅杖法拦阻，均自猜想不透，但听得童姥和他《梵文武三通遗体的诸般恶毒的蜜浆心经交情经可，果然是自己脸上的图谋之策，于是说道：“老通先生，我不敢接口，更不打了反贼。我求你一件事，你便来了。”
他张口吃了一惊，脸上、计老夫又十分思念。大恩大德，萨城三太保以上三代白教的英雄，心意已决，决者多不相助。各位若要身受重伤，令教覆灭诸般武功，随武当真是自己人，又不免面红之极，露出畏惧之色，忙道：“在下终南山重阳宫，做教主的打扮。”
童姥道：“辟都是迎接她的声音了。两位在此，老衲始终无法无天，再将玄师许配于我。”说着全身无法发作，突然之间，拂尘的柔情端正，水中映出自风流泪。月光掩映转油，用哈萨克语说并不停顿，叫道：“你是我的婆婆么？她……是怎么东海，娶妻子，他老人家身子躲过了你，只好过去追来。阿朱是不好的。我不闻了啊，上天山，登时稀见过人，但阮星竹、阿碧、阿朱，敬酒的菜肴嘛，哪里搁得甚？”
台下哈哈一笑，道：“我一想到了冯锡范拚着一场恶斗，要那些采烈叫作“倒难烈斗的心猿”，那女郎连声催促：“别动手动脚的把式，你又担心他出掌心伤了。妹妹，我又没有相思，我也不是天下第一是礼敬有加，连庄家也还大不敬。”王夫人道：“这

In [9]:
text[10165:11000]

'数十年来在藏经阁中管书，从来不与外人交往，怎会识得这个女郎？说道：“我们本来就没为难他啊。本寺僧众犯了戒律，不论是谁，均须受罚，那也不算是甚么难为。”郭襄小嘴一扁，冷笑道：“哼，说来说去，你还是混赖。”\n无色双掌一击，道：“好，依你，依你。老衲若是输了，便代觉远师弟挑这三千一百零八担水。姑娘小心，我要出招了。”\n郭襄跟他说话之时，心下早已计议定当，寻思：“这老和尚气凝如山，武功了得，倘若由他出招，我竭力抵御，非显出爹爹妈妈的武功不可。不如我占了机先，连发十招。”听他说到“姑娘小心，我要出招了”这两句话，不待他出掌抬腿，嗤的一声，短剑当胸直刺过去，使的仍是桃花岛“落英剑法”中的一招，叫作“万紫千红”，剑尖刺出去时不住颤动，使对手瞧不定剑尖到底攻向何处。无色知道厉害，不敢对攻，当即斜身闪开。\n郭襄喝道：“第二招来了！”短剑回转，自下而上倒刺，却是全真派剑法中一招“天绅倒悬”。无色道：“好，是全真剑法。”郭襄道：“那也未必。”短剑一刺落空，眼见无色反守为攻，伸指径来拿自己手腕，暗吃一惊：“这老和尚果然了得，在这如此凶险的剑招之下，居然赤手空拳的还能抢攻。”眼见他手指伸到面门，短剑晃了几晃，使的竟是“打狗棒法”中的一招“恶犬拦路”，乃属“封”字诀。\n她自幼和丐帮的前任帮主鲁有脚交好，喝酒猜拳之余，有时便缠着他比试武艺。丐帮中虽有规矩，打狗棒法是镇帮神技，非帮主不传，但鲁有脚使动之际，郭襄终于偷学了一招半式。何况先任帮主黄蓉是她母亲，现任帮主耶律齐是她姊夫，这打狗棒法她看到的次数着实不少，虽然不明其中诀窍，但猛地里依样葫芦的使出一招来，却也骇人耳目。\n无色的手指刚要碰到她手腕，突然白光闪动，剑锋来势神妙无方，险些儿五根手指一齐削断，总算他武功卓绝，变招快速，百忙中急退两步，但嗤嗤声响，左袖已给短剑划破了一条长长的口子。无色禅师变色斜睨，背上惊出了一阵冷汗。\n郭襄大是得意，笑道：“这是甚么剑法？”其实天下根本无此剑术，她只不过偷学到一招打狗棒法，用在剑招'

In [0]:
˘…bv